In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.stats import norm
from scipy import stats
import seaborn as sns
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy import signal

In [2]:
def divide_data_by_flow_direction(data):
    '''Dividing raw data from one fish into 2 dataframes with either left or right OMR flow'''
    data.columns = ['X_coord', 'Y_coord', 'heading_direction', \
                'cumulative_direction','beat_freq', 'beat_amp', \
                'tail_move?', 'timestamp', 'contrast_level', 'flow_direction']
    right = pd.DataFrame(data[data.flow_direction == 1])
    left = pd.DataFrame(data[data.flow_direction == 2])
    zero = pd.DataFrame(data[data.flow_direction == 0])

    return right, left, zero

def divide_data_by_contrast(data):
    '''Dividing raw data from one fish and one flow direction into contrast levels'''
    C_0 = pd.DataFrame(data[data.contrast_level == 0])
    C_01 = pd.DataFrame(data[data.contrast_level == 0.01])
    C_1 = pd.DataFrame(data[data.contrast_level == 0.1])
    C_2 = pd.DataFrame(data[data.contrast_level == 0.2])
    C_3 = pd.DataFrame(data[data.contrast_level == 0.3])
    C_5 = pd.DataFrame(data[data.contrast_level == 0.5])
    C_7 = pd.DataFrame(data[data.contrast_level == 0.7])
    C_10 = pd.DataFrame(data[data.contrast_level == 1])
    return C_0, C_01, C_1, C_2, C_3, C_5, C_7, C_10

def omr_preprocess(data):
    # remove timestamp and extras
    # keep x, y, heading, cumulative, timestamp
    data.columns = ['X_coord', 'Y_coord', 'heading_direction', \
                'cumulative_direction','beat_freq', 'beat_amp', \
                'tail_move?', 'timestamp', 'contrast_level', 'flow_direction']
    new = data.drop(columns = ['beat_freq', 'beat_amp','tail_move?', 'contrast_level', 'flow_direction'])


    # resetting index
    new = pd.DataFrame(new)
    new = new.set_index('timestamp').reset_index()


    # remove time points where there was an angle change of more than pi from one frame to another
    for i, row in new.iterrows():
        if i+1 == len(new):
            break
            # modify to add the exclusion zone
        if np.abs(new.at[i+1,'cumulative_direction']-new.at[i,'cumulative_direction']) >= 2.5:
            new.at[i+1, 'cumulative_direction'] = new.at[i,'cumulative_direction']


    # interpolating and normalising data to a fixed set of points
    interp = pd.DataFrame(columns=['timestamp','X_coord','Y_coord','heading_direction','cumulative_direction'])
    for column in new.columns:
        x = np.arange(0,len(new))
        y = new[column]
        f = interpolate.interp1d(x,y)

        x_new = np.arange(0,3000,1)
        y_new = f(x_new)
        interp[column] = y_new


    # setting first cumulative_angle to zero and ajdusting all others
    interp.iloc[:,4] -= interp.iloc[0,4]


    # calculating distance traveled between each timeframe
    # distance = sqrt((x2-x1)**2 + (y2-y1)**2)
    interp['distance_pts'] = 0
    for row in range(1,len(interp),1):
        distance = np.sqrt((interp['X_coord'][row]-interp['X_coord'][row-1])**2\
                            +(interp['Y_coord'][row]-interp['Y_coord'][row-1])**2)
        interp.iloc[row,5] = distance


    # cleaning the timestamps
    interp.insert(0, 'new_timestamp', range(1, 1 + len(interp)))
    interp = interp.drop(columns=['timestamp']).rename(columns={'new_timestamp':'timestamp'})
    interp['timestamp'] = interp['timestamp']/100

    return np.array(interp)

def combine_fish_data(*args):
    '''Combines preprocessed data from all fish into a 3D numpy array'''
    combined_fish = np.stack((args),axis=0)
    return combined_fish

def make_event_table(contrast): 
    fish_distance = contrast[:,-1]
    if np.sum(np.isnan(fish_distance)) > 0:
        np.nan_to_num(fish_distance,copy=False,nan=0)
    t = np.arange(0, 3000, 1)
    sig = fish_distance #composite signal
    sig_clean = sig #copy for later comparison
    minsignal, maxsignal = sig.min(), sig.max()
    widths = np.arange(1, 3000)
    cwtmatr = signal.cwt(sig, signal.ricker, widths)
    t = np.arange(0, 3000, 1)
    cwt_sig = cwtmatr[25,:] #composite signal
    signal_clean = sig #copy for later comparison
    minsignal, maxsignal = cwt_sig.min(), cwt_sig.max()
    ## Compute Fourier Transform
    n = len(t)
    fhat = np.fft.fft(cwt_sig, n) #computes the fft
    psd = fhat * np.conj(fhat)/n
    freq = (1/(1*n)) * np.arange(n) #frequency array
    idxs_half = np.arange(1, np.floor(n/2), dtype=np.int32) #first half index
    ## Filter out noise
    threshold = 1
    psd_idxs = psd > threshold #array of 0 and 1
    psd_clean = psd * psd_idxs #zero out all the unnecessary powers
    fhat_clean = psd_idxs * fhat #used to retrieve the signal
    signal_filtered = np.fft.ifft(fhat_clean) #inverse fourier transform
    fft_signal = signal_filtered.real
    fft_gauss = gaussian_filter1d(fft_signal,sigma=5)
    fish_distance_2 = []
    for frame in fft_gauss:
        if frame<=1:
            frame=0
        fish_distance_2.append(frame)
    fft_gauss = fish_distance_2
    
    # fiter out events mid-beginning and mid-end 
    if fft_gauss[0] != 0:
        zero_start = fft_gauss.index(0)
        fft_gauss[:zero_start] = [0]*(zero_start)
    if fft_gauss[-1] != 0:
        end_zero = -(fft_gauss[::-1].index(0))
        fft_gauss[end_zero:] = [0]*(-end_zero) 
    

    indices = []
    for val in fft_gauss:
        if val == 0:
            indices.append(0)
        else:
            indices.append(fft_gauss.index(val))


    df = pd.DataFrame(fft_gauss,columns=['fft_gauss'])
    df['ind'] = indices
    df['init_cum'] = contrast[:,4]
    df['init_dist'] = contrast[:,5]
    df['event'] = 0


    counter = 1 
    for index, row in df.iterrows():  
        if row['fft_gauss'] != 0:
            df.at[index,'event'] = counter
            if df.at[index+1,'fft_gauss'] == 0:
                counter += 1


    event_df = pd.DataFrame(columns=['event','duration_s','delta_theta_rad','distance','start_ind','end_ind','latency_s'])
    event_df['event'] = df['event'].unique()[1:]

    event_df['duration_s'] = [((df['event']==event).sum())/100 for event in (df['event']).unique()[1:]]

    event_df['start_ind'] = [int(df[df['event']==event].reset_index().iloc[0][0]) for event in (df['event']).unique()[1:]]
    event_df['end_ind'] = [int(df[df['event']==event].reset_index().iloc[-1][0]) for event in (df['event']).unique()[1:]]

    event_df['delta_theta_rad'] = [df['init_cum'][r['end_ind']]-df['init_cum'][r['start_ind']] \
                                   for i, r in event_df.iterrows()]

    event_df['latency_s'] = [None if i-1 == -1 else (event_df['start_ind'][i]-event_df['end_ind'][i-1])/100 \
                             for i, r in event_df.iterrows()]

    # event_df['distance'] = ADD all frames per event except first one
    event_df['distance'] = [df[df['event']==ev]['init_dist'].sum() - df[df['event']==ev]['init_dist'].reset_index()\
                            ['init_dist'][0] for ev in df['event'].unique()[1:]]

    event_df = event_df.drop(columns=['start_ind','end_ind'])
    
    return event_df

In [3]:
fish = pd.read_csv("../raw_data/AM/Met/xy_hc_fai_tstp_Me_1_AM.csv")

In [4]:
R, L, Z = divide_data_by_flow_direction(fish)
L0, LC1, LC10, LC20, LC30, LC50, LC70, LC100 = divide_data_by_contrast(L)
R0, RC1, RC10, RC20, RC30, RC50, RC70, RC100 = divide_data_by_contrast(R)

In [5]:
for flow in ['L','R']:
    for c in ['1','10','20','30','50','70','100']:
        exec(f'{flow}C{c} = omr_preprocess({flow}C{c})')

In [6]:
# multiply right flows by -1
RC1[:,4] = abs(RC1[:,4])
RC10[:,4] = abs(RC10[:,4])
RC20[:,4] = abs(RC20[:,4])
RC30[:,4] = abs(RC30[:,4])
RC50[:,4] = abs(RC50[:,4])
RC70[:,4] = abs(RC70[:,4])
RC100[:,4] = abs(RC100[:,4])

In [7]:
# combine left and right flows
C1 = np.stack((RC1,LC1))
C10 = np.stack((RC10,LC10))
C20 = np.stack((RC20,LC20))
C30= np.stack((RC30,LC30))
C50 = np.stack((RC50,LC50))
C70 = np.stack((RC70,LC70))
C100 = np.stack((RC100,LC100))

In [8]:
# make fish list
C1.shape

(2, 3000, 6)

In [9]:
make_event_table(C100[1])

,event,duration_s,delta_theta_rad,distance,latency_s
0,1,0.49,0.662323,36.940254,NaN
1,2,0.49,0.564440,50.204296,3.12
2,3,0.44,0.600491,44.106592,2.24
3,4,0.45,1.266456,35.768263,17.41
4,5,0.45,0.566549,39.231773,1.16


In [10]:
df_names = ['am_me_1','am_me_10','am_me_20','am_me_30','am_me_50','am_me_70','am_me_100']
dataframes = [C1,C10,C20,C30,C50,C70,C100]

In [11]:
full_event_table = pd.DataFrame(columns=['event_n','fish_n','event','duration_s','delta_theta_rad','distance',\
                                         'latency_s','time','medium','contrast'])

fish_counter = 1
for name,df in zip(df_names,dataframes):
    for fish in df:
        event_df = make_event_table(fish)
        event_df['time'] = name.split('_')[0]
        event_df['medium'] = name.split('_')[1]
        event_df['contrast'] = int(name.split('_')[2])
        event_df['event_n'] = 0
        event_df['fish_n'] = fish_counter
        fish_counter += 1
        full_event_table = pd.concat([full_event_table,event_df])
        
full_event_table['event_n'] = np.arange(1,len(full_event_table)+1,1)
full_event_table = full_event_table.set_index('event_n')

full_event_table

,fish_n,event,duration_s,delta_theta_rad,distance,latency_s,time,medium,contrast
event_n,,,,,,,,,
1,1,1,0.38,0.1726,22.86368,NaN,am,me,1
2,1,2,0.41,0.35736,26.607389,4.03,am,me,1
3,1,3,0.45,0.03132,27.925136,2.32,am,me,1
4,1,4,0.53,0.4523,42.705481,1.64,am,me,1
5,1,5,0.45,-0.33232,31.895262,3.22,am,me,1
...,...,...,...,...,...,...,...,...,...
75,14,1,0.49,0.662323,36.940254,NaN,am,me,100
76,14,2,0.49,0.56444,50.204296,3.12,am,me,100
77,14,3,0.44,0.600491,44.106592,2.24,am,me,100


In [12]:
am_me_C1_events = full_event_table[full_event_table['contrast']==1]
am_me_C10_events = full_event_table[full_event_table['contrast']==10]
am_me_C20_events = full_event_table[full_event_table['contrast']==20]
am_me_C30_events = full_event_table[full_event_table['contrast']==30]
am_me_C50_events = full_event_table[full_event_table['contrast']==50]
am_me_C70_events = full_event_table[full_event_table['contrast']==70]
am_me_C100_events = full_event_table[full_event_table['contrast']==100]

In [13]:
am_me_C1_events

,fish_n,event,duration_s,delta_theta_rad,distance,latency_s,time,medium,contrast
event_n,,,,,,,,,
1,1,1,0.38,0.1726,22.86368,NaN,am,me,1
2,1,2,0.41,0.35736,26.607389,4.03,am,me,1
3,1,3,0.45,0.03132,27.925136,2.32,am,me,1
4,1,4,0.53,0.4523,42.705481,1.64,am,me,1
5,1,5,0.45,-0.33232,31.895262,3.22,am,me,1
6,1,6,0.44,0.02165,31.376512,1.32,am,me,1
7,1,7,0.43,0.46432,32.288186,7.82,am,me,1
8,2,1,0.52,0.97538,46.118109,NaN,am,me,1
9,2,2,0.43,-0.00389,29.471991,1.57,am,me,1


In [14]:
am_me_C1_events.loc[(am_me_C1_events['fish_n']==1)]['distance'].sum()

215.66164614767666

In [15]:
stats.sem(am_me_C1_events.loc[(am_me_C1_events['fish_n']==1)]['latency_s'][1:])

0.9756311347589883

In [16]:
def make_fish_list():    
    fish_list = pd.DataFrame(columns=['fish_n','bouts_Hz','duration_avg','duration_sem','latency_avg',\
                                      'latency_sem','distance_avg','distance_sem','total_distance'])


    fish_list['fish_n'] = [fish for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['bouts_Hz'] = [(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish) & \
                            (am_me_C1_events['event']==am_me_C1_events['event'].max())].values[0][1])/30 \
                             for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['duration_avg'] = [np.mean(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['duration_s']) \
                                 for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['duration_sem'] = [stats.sem(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['duration_s']) \
                                 for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['latency_avg'] = [np.mean(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['latency_s']) \
                                for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['latency_sem'] = [stats.sem(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['latency_s'][1:]) \
                               for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['distance_avg'] = [np.mean(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['distance']) \
                                for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['distance_sem'] = [stats.sem(am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['distance']) \
                               for fish in am_me_C1_events['fish_n'].unique()]
    fish_list['total_distance'] = [am_me_C1_events.loc[(am_me_C1_events['fish_n']==fish)]['distance'].sum() \
                                  for fish in am_me_C1_events['fish_n'].unique()]
    # fish_list['final_cumulative_angle'] = None

    return fish_list

In [18]:
make_fish_list()

,fish_n,bouts_Hz,duration_avg,duration_sem,latency_avg,latency_sem,distance_avg,distance_sem,total_distance
0,1,0.233333,0.441429,0.017516,3.391667,0.975631,30.808807,2.361974,215.661646
1,2,0.233333,0.468571,0.016822,3.220000,0.579413,42.282218,6.168296,295.975525
